In [2]:
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd
import os
import sys
from osgeo import gdal,ogr, osr
import xarray
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

## 1. Daymet forcing data processing

In [3]:
def to_tif(array, x, y, out_path, date, var, proj,shapefile):
    rows, cols = array.shape
    xmin, pixel_size, ymax = x.min(), x[1]-x[0], y.max()

    geotransform = (xmin, pixel_size, 0, ymax, 0, -pixel_size)

    output_tif_file = f"{out_path}{var}/{date}_{var}.tif"
    driver = gdal.GetDriverByName("GTiff")

    dataset = driver.Create(output_tif_file, cols, rows, 1, gdal.GDT_Float32)

    projection = osr.SpatialReference()
    projection.SetProjCS(proj.grid_mapping_name)
    projection.SetLCC(proj.standard_parallel[0], proj.standard_parallel[1], proj.latitude_of_projection_origin, proj.longitude_of_central_meridian,proj.false_easting, proj.false_northing)
    projection.SetGeogCS("WGS 84",
                         "WGS 84",
                         "WGS 84",
                         proj.semi_major_axis,
                         proj.inverse_flattening)

    dataset.SetGeoTransform(geotransform)
    dataset.SetProjection(projection.ExportToWkt())
    dataset.GetRasterBand(1).WriteArray(array)
    dataset = None

    input_ds = gdal.Open(output_tif_file)
    output_ds = gdal.Warp(output_tif_file, input_ds, cutlineDSName=shapefile, cropToCutline=True, dstNodata=0)

    input_ds = None
    output_ds = None
    return

In [ ]:
variables = ["dayl","prcp","srad","tmax","tmin","vp"]
years = ["2020","2021","2022","2023"]

data_path = "/Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/5_Forcing_data/1_original_data/"
out_path = "/Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/5_Forcing_data/2_clipped_us_extent/"
shapefile = "/Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/5_Forcing_data/2_clipped_us_extent/0_us_shapefile/us_mainland.shp"

for var in variables:
    os.makedirs(f"{out_path}{var}", exist_ok=True)
    for yrs in years:
        file_name = f"{data_path}daymet_v4_daily_na_{var}_{yrs}.nc"
        file = xarray.open_dataset(file_name)
        data = file[var]
        
        x = file["x"].values
        y = file["y"].values
        proj = file["lambert_conformal_conic"]
        for i in range(data.shape[0]):
            data_array = data[i,:,:]
            date = str(data_array["time"].values.astype('datetime64[D]'))
            print(var, date)
            tif_array = data_array.values
            to_tif(tif_array, x, y, out_path, date, var, proj,shapefile)

## 2. Convert projection to geographic coordinate

In [ ]:
data_path = "/Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/5_Forcing_data/2_clipped_us_extent/"
out_path = "/Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/5_Forcing_data/3_clipped_forcing_WGS84/"

variables = ["dayl","prcp","srad","tmax","tmin","vp"]

for var in  variables:
    file_name = os.listdir(f"{data_path}{var}")
    file_name = [x for x in file_name if ".tif" in x and ".aux.xml" not in x and "._" not in x]
    for file in file_name:
        input_file = f"{data_path}{var}/{file}"
        output_file = f"{out_path}{var}/{file}"
        dataset = gdal.Open(input_file)
        print(var,datetime.now().strftime("%Y-%m-%d %H:%M:%S"),file,"finished")
        gdal.Warp(output_file, dataset, dstSRS='EPSG:4326')
        dataset = None

## 3. clip using NEON site extent

In [ ]:
def read_tif(tif_file):
    dataset = gdal.Open(tif_file)
    cols = dataset.RasterXSize
    rows = dataset.RasterYSize
    im_proj = (dataset.GetProjection())
    im_Geotrans = (dataset.GetGeoTransform())
    im_data = np.moveaxis(dataset.ReadAsArray(0, 0, cols, rows), 0, -1)
    return im_data, im_Geotrans, im_proj,rows, cols

prisma_path = "/Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/2_PRISMA_L2D/2_PRISMA_L2D_tif_2020_2023/"
forcing_path = "/Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/5_Forcing_data/3_clipped_forcing_WGS84/"
out_path = "/Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/5_Forcing_data/4_clipped_NEON_extent/"

variables = ["dayl","LAI","prcp","srad","tmax","tmin","vp"]
site_list = ['D01_BART','D01_HARV','D02_SCBI','D03_OSBS','D07_MLBS','D07_ORNL','D08_TALL',
             'D10_CPER','D13_MOAB','D14_JORN','D14_SRER','D16_WREF','D19_BONA','D19_HEAL']

for site in site_list:
    path = f"{prisma_path}{site}"
    file_list = os.listdir(path)
    file_list = [x for x in file_list if "LATLON.tif" in x]

    ul_x_all,lr_y_all, lr_x_all, ul_y_all = [],[],[],[]
    for file in file_list:
        im_data, im_Geotrans, im_proj,rows, cols = read_tif(f"{path}/{file}")
        ul_x, lr_y, lr_x, ul_y = im_data[0,0,1],im_data[-1,-1,0],im_data[-1,-1,1],im_data[0,0,0]
        ul_x_all.append(ul_x)
        lr_y_all.append(lr_y)
        lr_x_all.append(lr_x)
        ul_y_all.append(ul_y)
    ul_x, lr_y, lr_x, ul_y = min(ul_x_all),min(lr_y_all), max(lr_x_all), max(ul_y_all)
    ul_x, lr_y, lr_x, ul_y = ul_x-0.2, lr_y-0.2, lr_x+0.2, ul_y+0.2
    print(site, ul_x, lr_y, lr_x, ul_y)
    
    for var in variables:
        var_in_folder = f"{forcing_path}{var}"
        var_out_folder = f"{out_path}{var}"
        forcing_files = os.listdir(var_in_folder)
        forcing_files = [x for x in forcing_files if ".tif" in x and ".aux.xml" not in x and "._" not in x]
        os.makedirs(f"{var_out_folder}/{site}", exist_ok=True)
        
        for forcing_file in forcing_files:
            input_tif = f"{var_in_folder}/{forcing_file}"
            output_tif =f"{var_out_folder}/{site}/{forcing_file}"
            gdal.Warp(output_tif, input_tif, format = 'GTiff', outputBounds=(ul_x, lr_y, lr_x, ul_y))
            input_tif = None
            output_tif = None
            print(site,var,datetime.now().strftime("%Y-%m-%d %H:%M:%S"),forcing_file,"finished")
            
            ### transfer the projection of clipped land use data
            in_tif = f"{var_out_folder}/{site}/{forcing_file}"
            out_tif = f"{var_out_folder}/{site}/{forcing_file}"
            input_ds = gdal.Open(in_tif)
            output_ds = gdal.Warp(out_tif, input_ds, dstSRS=im_proj)
            input_ds = None
            output_ds = None